In [1]:
import pandas as pd
import numpy as np
from numpy import random 
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

The functions below add several numerical features to my dataset. The first function, all_time_win_ratio, maps every team's all time win ratio to the opponent column. The second function, win_ratio_head2head, maps the selected team's head to head win ratio vs each opponent to the opponent column. The third function, add_league_position, maps the league position of each team in the previous season to the opponent column, with newly promoted teams getting a blanket 20th position. the fourth function, home_away_numerical adds a 1 or 0 based on whether the indexed team is home or away. The 5th function, win_streak counts the number of succesive wins a team has achieved and resets to 0 if a loss or tie is recorded. Functions 6 and 7 are for producing figures for my report. The final function adds the points column (my target). This is whether the indexed team has won, lost, or drawn the game in question.

In [2]:
#### Function 1
def all_time_win_ratio(df):
    team_list = df.HomeTeam.unique()
    all_time_ratio = {}
    for team in team_list:
        all_games = df.loc[(slice(None),team),:]
        win_ratio = len(all_games[all_games['Points'] == 3])/len(all_games)
        all_time_ratio[team] = win_ratio
    df['opp_win_ratio'] = df['opponent'].map(all_time_ratio)
    return df

#### Function 2
def win_ratio_head2head(df):
    team_list = df.HomeTeam.unique()
    chunks = []
    for team in team_list:
        team_games = df[df['team'] == team]
        head2head = {}
        opponents = team_games['opponent'].unique()
        for opponent in opponents:
            wins = len(team_games.loc[(team_games['opponent']==opponent) & (team_games['Points'] == 3)])
            total = len(team_games.loc[(team_games['opponent'] == opponent)])
            if wins > 0:
                win_ratio = wins/total
            else:
                win_ratio = 0
            head2head[opponent] = win_ratio
        team_games['head2head_win_ratio'] = team_games['opponent'].map(head2head)
        chunks.append(team_games)
    dataframe = pd.concat(chunks,ignore_index=True)
    return dataframe

#### Function 3
def add_league_position(df1,df2):
    seasons_list1 = double_df['Season'].unique()
    seasons_list2 = seasons.year.unique()
    seasons_offset = [x for x in seasons_list1 if x in seasons_list2]
    seasons_offset.remove('1993-94')
    seasons_list1 = double_df['Season'].unique()
    seasons_list2 = seasons.year.unique()
    seasons_list = [x for x in seasons_list1 if x in seasons_list2]
    seasons_list.remove('2016-17')
    chunk_list = []
    for i in range(len(seasons_offset)):
        current = df2[df2['year']==seasons_list[i]]
        teams = current['Team']
        pos = current['Pos']
        dictionary = dict(zip(teams,pos))
        chunk = df1[df1['Season']==seasons_offset[i]]
        chunk['Opp_League_Finish_Last_Year'] = chunk['opponent'].map(dictionary)
        chunk_list.append(chunk)
    dataframe = pd.concat(chunk_list,ignore_index=True)
    return dataframe

#### Function 4
def home_away_numerical(df):
    team_list = df.HomeTeam.unique()
    chunks = []
    for team in team_list:
        all_games = df[df['team']==team]
        all_games['HomeAway'] = 1
        all_games.loc[all_games['AwayTeam']==team,'HomeAway'] = 0
        chunks.append(all_games)
    dataframe = pd.concat(chunks,ignore_index=True)
    return dataframe

#### Function 5
def win_streak(df):
    team_list = df['HomeTeam'].unique()
    chunk_list = []
    for team in team_list:
        team_games = double_df.loc[(slice(None),team),:]
        streak_list = []
        streak = 0
        for (season, team, date), data in team_games.iterrows():
            if data['Points'] == 3:
                streak_list.append(streak)
                streak += 1
            elif data['Points'] != 3: 
                streak_list.append(streak)
                streak = 0
        team_games['Streak'] = streak_list
        chunk_list.append(team_games)
    dataframe = pd.concat(chunk_list)
    return dataframe

#### Function 6
def home_away_points(df, season):
    team_list = df.loc[season]['HomeTeam'].unique()
    home_points= []
    away_points = []
    for team in team_list:
        team_df = df.loc[(season,team)]
        home = team_df['HomeTeam'] == team
        away = team_df['AwayTeam'] == team
        home_points.append(team_df.loc[home,'Points'].mean().round(2))
        away_points.append(team_df.loc[away, 'Points'].mean().round(2))
    dataframe = pd.DataFrame({'Teams':team_list,'Mean Home Points':home_points,'Mean Away Points' : away_points})
    dataframe.set_index('Teams', inplace=True)
    dataframe.plot(kind='Bar',title='Mean Home and Away Points in ' + season)

#### Function 7    
def cumulative_points(df,season):
    team_list = df.loc[season]['HomeTeam'].unique()
    most_least_list = []
    most_least_dict = {}
    for team in team_list:
        last_year = df.loc[(season,team)]
        most_least_list.append(last_year['Points'].sum())
        most_least_dict[last_year['Points'].sum()] = team
    most_least_list = sorted(most_least_list)
    best_team = df.loc[(season,most_least_dict[most_least_list[-1]])]
    mid_team = df.loc[(season,most_least_dict[most_least_list[9]])]
    worst_team = df.loc[(season,most_least_dict[most_least_list[0]])]
    best_team = best_team.reset_index(drop=True)
    mid_team = mid_team.reset_index(drop=True)
    worst_team = worst_team.reset_index(drop=True)
    best_cum_points = best_team['Points'].cumsum()
    best_cum_randoms = best_team['Random Points'].cumsum()
    mid_cum_points = mid_team['Points'].cumsum()
    mid_cum_randoms = mid_team['Random Points'].cumsum()
    worst_cum_points = worst_team['Points'].cumsum()
    worst_cum_randoms = worst_team['Random Points'].cumsum()
    best_cum_points.plot(color='blue',label='Top Team Real Results')
    best_cum_randoms.plot(color='red',label='Top Team Random Results')
    mid_cum_points.plot(color='green', label = 'Mid Table Team Real Results')
    mid_cum_randoms.plot(color='yellow', label='Mid Table Team Random Results')
    worst_cum_points.plot(color='brown',label='Bottom Team Real Results')
    worst_cum_randoms.plot(color='black',label='Bottom Team Random Results')
    plt.xlabel('Games')
    plt.ylabel('Points')
    plt.title("Point Total from " + season + ': Expected vs. Actual')
    plt.legend(loc='upper left')
    plt.tight_layout()
    plt.show()

#### Function 8
def results_and_points(df, team_list):
    for team in team_list:    
        real_win = ((df['team'] == team) & (df['HomeTeam'] == team) & (df['FTR'] =='H')) |((df['team'] == team) & (df['AwayTeam'] ==team) & (df['FTR'] =='A'))
        real_tie = ((df['team'] == team) & (df['HomeTeam'] == team) & (df['FTR'] =='D')) |((df['team'] == team) & (df['AwayTeam'] ==team) & (df['FTR'] =='D'))
        real_loss = ((df['team'] == team) & (df['HomeTeam'] == team) & (df['FTR'] =='A')) |((df['team'] == team) & (df['AwayTeam'] ==team) & (df['FTR'] =='H'))
        random_win = ((df['team'] == team) & (df['HomeTeam'] == team) & (df['randomResult'] =='H')) |((df['team'] == team) & (df['AwayTeam'] ==team) & (df['randomResult'] =='A'))
        random_tie = ((df['team'] == team) & (df['HomeTeam'] == team) & (df['randomResult'] =='D')) |((df['team'] == team) & (df['AwayTeam'] ==team) & (df['randomResult'] =='D'))
        random_loss = ((df['team'] == team) & (df['HomeTeam'] == team) & (df['randomResult'] =='A')) |((df['team'] == team) & (df['AwayTeam'] ==team) & (df['randomResult'] =='H'))
        df.loc[real_win,'Points']=3
        df.loc[real_tie,'Points']=1
        df.loc[real_loss, 'Points']=0
        df.loc[random_win, 'Random Points'] = 3
        df.loc[random_tie, 'Random Points'] = 1
        df.loc[random_loss, 'Random Points'] = 0
    return df
    

In [3]:
#Loading in a dataframe of match results and season results and cleaning columns
original_df = pd.read_csv('EPL_Match_Results.csv')
original_df = original_df[['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','Season']]
original_df['Date'] = pd.to_datetime(original_df['Date'],dayfirst=True)

#Adding random results with a random integer and changing all the 2's to 3's
original_df['randomResult'] = np.random.randint(0,3,len(original_df))
away_win = original_df['randomResult'] == 0
draw = original_df['randomResult'] == 1
home_win = original_df['randomResult'] == 2               
original_df.loc[home_win,'randomResult'] = 'H'
original_df.loc[draw, 'randomResult'] = 'D'
original_df.loc[away_win, 'randomResult'] = 'A'

#Loading in a second dataframe, seasons, to add league position as a feature in my original_df. Team names are recorded
#differently than in my original_df, as were season titles, so a lot of strings had to be changed.
seasons = pd.read_csv('seasons_result.csv',index_col= 'year')
seasons = seasons.loc['2016/2017':'1993/1994'][['Team','W','L','D','F','A','Pts','Pos']]
seasons = seasons.reset_index()
seasons['year'] = seasons['year'].str.slice_replace(4,7,'-')
seasons['Team'] = seasons['Team'].str.replace('Manchester United', 'Man United')
seasons['Team'] = seasons['Team'].str.replace('Manchester City', 'Man City')
seasons['Team'] = seasons['Team'].str.replace('Tottenham Hotspurs', 'Tottenham')
seasons['Team'] = seasons['Team'].str.replace('West Bromwich Albion', 'West Brom')
seasons['Team'] = seasons['Team'].str.replace('Birmingham City', 'Birmingham')
seasons['Team'] = seasons['Team'].str.replace('Blackburn Rovers', 'Blackburn')
seasons['Team'] = seasons['Team'].str.replace('Bolton Wanderers', 'Bolton')
seasons['Team'] = seasons['Team'].str.replace('Bradford City', 'Bradford')
seasons['Team'] = seasons['Team'].str.replace('Cardiff City', 'Cardiff')
seasons['Team'] = seasons['Team'].str.replace('Charlton Athletic', 'Charlton')
seasons['Team'] = seasons['Team'].str.replace('Coventry City', 'Coventry')
seasons['Team'] = seasons['Team'].str.replace('Derby County', 'Derby')
seasons['Team'] = seasons['Team'].str.replace('Hull City', 'Hull')
seasons['Team'] = seasons['Team'].str.replace('Ipswich Town', 'Ipswich')
seasons['Team'] = seasons['Team'].str.replace('Leeds United', 'Leeds')
seasons['Team'] = seasons['Team'].str.replace('Leicester City', 'Leicester')
seasons['Team'] = seasons['Team'].str.replace('Newcastle United', 'Newcastle')
seasons['Team'] = seasons['Team'].str.replace('Norwich City', 'Norwich')
seasons['Team'] = seasons['Team'].str.replace('Nottingham Forest', "Nott'm Forest")
seasons['Team'] = seasons['Team'].str.replace('Queens Park Rangers', 'QPR')
seasons['Team'] = seasons['Team'].str.replace('Oldham Athletic','Oldham')
seasons['Team'] = seasons['Team'].str.replace('Sheffield Wednesday', 'Sheffield Weds')
seasons['Team'] = seasons['Team'].str.replace('Stoke City', 'Stoke')
seasons['Team'] = seasons['Team'].str.replace('Swansea City', 'Swansea')
seasons['Team'] = seasons['Team'].str.replace('Swindon Town', 'Swindon')
seasons['Team'] = seasons['Team'].str.replace('West Bromwich Albion', 'West Brom')
seasons['Team'] = seasons['Team'].str.replace('West Ham United','West Ham')
seasons['Team'] = seasons['Team'].str.replace('Wolverhampton Wanderers','Wolves')
seasons['year'] = seasons['year'].str.replace('2002-','2002-03')
seasons['year'] = seasons['year'].str.replace('2003-','2003-04')
seasons['year'] = seasons['year'].str.replace('2004-','2004-05')
seasons['year'] = seasons['year'].str.replace('2005-','2005-06')

team_list = original_df.HomeTeam.unique()
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)
print(original_df.head(10))

        Date          HomeTeam        AwayTeam  FTHG  FTAG FTR   Season randomResult
0 1993-08-14           Arsenal        Coventry     0     3   A  1993-94            A
1 1993-08-14       Aston Villa             QPR     4     1   H  1993-94            D
2 1993-08-14           Chelsea       Blackburn     1     2   A  1993-94            A
3 1993-08-14         Liverpool  Sheffield Weds     2     0   H  1993-94            A
4 1993-08-14          Man City           Leeds     1     1   D  1993-94            A
5 1993-08-14         Newcastle       Tottenham     0     1   A  1993-94            D
6 1993-08-14            Oldham         Ipswich     0     3   A  1993-94            D
7 1993-08-14  Sheffield United         Swindon     3     1   H  1993-94            D
8 1993-08-14       Southampton         Everton     0     2   A  1993-94            H
9 1993-08-14          West Ham       Wimbledon     0     2   A  1993-94            A


The following cell turns the dataframe I originally loaded into the format I need for analyzing matches. I had to copy the original and make new columns for both so that when I concatenated them together I could have all of a team's games represented in order by season. This cell also applies all the functions described above to obtain all my numerical features.

In [4]:
pd.options.mode.chained_assignment = None
original_df1 = original_df.copy()
original_df1['team'] = original_df1['HomeTeam']
original_df1['opponent'] = original_df1['AwayTeam']
original_df['team'] = original_df1['AwayTeam']
original_df['opponent'] = original_df1['HomeTeam']
double_df = pd.concat([original_df,original_df1])
double_df = results_and_points(double_df, team_list)
double_df = win_ratio_head2head(double_df)
double_df = home_away_numerical(double_df)
double_df = add_league_position(double_df,seasons)
double_df.set_index(['Season','team','Date'], inplace=True)
double_df.sort_index(inplace=True, level = [0,1,2])
double_df = all_time_win_ratio(double_df)
double_df = win_streak(double_df)
double_df.sort_index(inplace=True, level = [0,1,2])
double_df['Opp_League_Finish_Last_Year'].fillna(20,inplace=True)
double_df['Result'] = 'Win'
double_df.loc[double_df['Points'] == 1, 'Result'] = 'Draw'
double_df.loc[double_df['Points']== 0, 'Result'] = 'Loss'
double_df.head()

HomeTeam   AwayTeam  FTHG  FTAG FTR   ...   HomeAway Opp_League_Finish_Last_Year  opp_win_ratio  Streak  Result
Season  team    Date                                               ...                                                                      
1994-95 Arsenal 1994-08-20    Arsenal   Man City     3     0   H   ...          1                        16.0       0.436923       0     Win
                1994-08-23      Leeds    Arsenal     1     0   H   ...          0                         5.0       0.414062       1    Loss
                1994-08-28  Liverpool    Arsenal     3     0   H   ...          0                         8.0       0.510714       0    Loss
                1994-08-31    Arsenal  Blackburn     0     0   D   ...          1                         2.0       0.354575       0    Draw
                1994-09-10    Norwich    Arsenal     0     0   D   ...          0                        12.0       0.242268       0    Draw

[5 rows x 15 columns]

In [5]:
train = double_df.loc['1994-95':'2015-16']
X_train = train.drop(['opponent','FTHG','FTAG','HomeTeam','AwayTeam', 'Random Points','randomResult','FTR','Points','Result'],axis = 1)
y_train = train['Result']
test = double_df.loc['2016-17']
X_test = test.drop(['opponent','FTHG','FTAG','HomeTeam','AwayTeam', 'Random Points','randomResult','FTR','Points','Result'],axis = 1)
y_test = test['Result']
logreg_grid = LogisticRegression(multi_class='multinomial',solver = 'lbfgs',max_iter=1000)
params = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
CLF = GridSearchCV(logreg_grid,params,cv = 5,scoring='accuracy')
CLF.fit(X_train,y_train)
CLF.best_params_

In [6]:
logreg = LogisticRegression(multi_class='multinomial',solver = 'lbfgs',max_iter=1000, C = 0.1)
logreg.fit(X_train,y_train)
logreg.score(X_test,y_test)

0.5947368421052631

In [7]:
y_pred = logreg.predict(X_test)
rep = classification_report(y_test,y_pred)
print(rep)
print(logreg.coef_)
print(X_test.columns)

              precision    recall  f1-score   support

        Draw       0.41      0.08      0.14       168
        Loss       0.60      0.72      0.66       296
         Win       0.60      0.76      0.67       296

   micro avg       0.59      0.59      0.59       760
   macro avg       0.54      0.52      0.49       760
weighted avg       0.56      0.59      0.55       760

[[-1.14704687 -0.02145869  0.00653336 -0.82418705  0.03736893]
 [-1.93267458 -0.63671924 -0.02204734  0.5614991  -0.07155235]
 [ 3.07972146  0.65817793  0.01551397  0.26268795  0.03418342]]
Index(['head2head_win_ratio', 'HomeAway', 'Opp_League_Finish_Last_Year', 'opp_win_ratio', 'Streak'], dtype='object')


In [14]:
train = double_df.loc['1994-95':'2015-16']
X_train = train.drop(['opponent','FTHG','FTAG','HomeTeam','AwayTeam', 'Random Points','randomResult','FTR','Points',
                      'Result'],axis = 1)
y_train = train['Result']
test = double_df.loc['2016-17']
X_test = test.drop(['opponent','FTHG','FTAG','HomeTeam','AwayTeam', 'Random Points','randomResult','FTR','Points',
                    'Result'],axis = 1)
y_test = test['Result']
rf_grid = RandomForestClassifier(max_features='sqrt')
params = {'n_estimators':[250,500,750],'max_depth': np.linspace(7,13,6)}
clf = GridSearchCV(rf_grid,params,cv = 5,scoring='accuracy')
clf.fit(X_train,y_train)
print(clf.best_params_,clf.best_score_)

In [15]:
rf = RandomForestClassifier(max_features = 'sqrt', n_estimators = 250, max_depth = 7 )
rf.fit(X_train,y_train)
rf.score(X_test,y_test)

0.5802631578947368

In [16]:
forest_pred =rf.predict(X_test)
forest_rep = classification_report(y_test,forest_pred)
print(forest_rep)

              precision    recall  f1-score   support

        Draw       0.34      0.08      0.13       168
        Loss       0.60      0.68      0.63       296
         Win       0.59      0.77      0.67       296

   micro avg       0.58      0.58      0.58       760
   macro avg       0.51      0.51      0.48       760
weighted avg       0.54      0.58      0.53       760



In [39]:
two_target = double_df.copy()
draws = two_target['Result']=='Draw'
two_target.loc[draws,'Result'] = 'Loss'
loss_draw = two_target['Result'] == 'Loss'
two_target.loc[loss_draw,'Result'] = 'Loss/Draw'

train = two_target.loc['1994-95':'2015-16']
X_train = train.drop(['opponent','FTHG','FTAG','HomeTeam','AwayTeam', 'Random Points','randomResult','FTR','Points','Result'],axis = 1)
y_train = train['Result']
test = two_target.loc['2016-17']
X_test = test.drop(['opponent','FTHG','FTAG','HomeTeam','AwayTeam', 'Random Points','randomResult','FTR','Points','Result'],axis = 1)
y_test = test['Result']

In [40]:
logreg = LogisticRegression(solver = 'lbfgs',max_iter=1000, C = 0.1)
logreg.fit(X_train,y_train)
logreg.score(X_test,y_test)
y_pred = logreg.predict(X_test)
twotest_report = classification_report(y_test,y_pred)
print(twotest_report)

              precision    recall  f1-score   support

   Loss/Draw       0.75      0.86      0.80       464
         Win       0.71      0.56      0.63       296

   micro avg       0.74      0.74      0.74       760
   macro avg       0.73      0.71      0.71       760
weighted avg       0.74      0.74      0.73       760



In [44]:
top_teams_model = two_target.loc[(slice(None),['Arsenal','Chelsea','Man United','Liverpool','Tottenham','Man City']),:]
train_topteams = top_teams_model.loc['1994-95':'2015-16']
X_train_topteams = train_topteams.drop(['FTHG','FTAG','HomeTeam','AwayTeam', 'Random Points','randomResult','FTR','Points','Result','opponent'],axis = 1)
y_train_topteams = train_topteams['Result']
test_topteams = top_teams_model.loc['2016-17']
X_test_topteams = test_topteams.drop(['FTHG','FTAG','HomeTeam','AwayTeam', 'Random Points','randomResult','FTR','Points','Result','opponent'],axis = 1)
y_test_topteams = test_topteams['Result']
logreg_top = LogisticRegression(solver = 'lbfgs',max_iter=1000, C = 0.1)
logreg_top.fit(X_train_topteams,y_train_topteams)
y_pred_topteams = logreg_top.predict(X_test_topteams)
topteams_report = classification_report(y_test_topteams,y_pred_topteams)
print(topteams_report)

              precision    recall  f1-score   support

   Loss/Draw       0.55      0.55      0.55        86
         Win       0.73      0.73      0.73       142

   micro avg       0.66      0.66      0.66       228
   macro avg       0.64      0.64      0.64       228
weighted avg       0.66      0.66      0.66       228



In [192]:
bets = pd.read_csv('E0.csv')
bets = bets[['Date','HomeTeam','AwayTeam','FTR','B365H','B365A']]
team_list = test.HomeTeam.unique()
team_list.sort()

In [215]:
def did_I_make_money(team_list,predictions,betting_odds):
    teams = []
    total_bets = []
    money_made_or_lost = []
    x=0
    x1=38
    for team in team_list:
        team_odds = ((betting_odds['HomeTeam'] ==team)|(bets['AwayTeam']==team))
        team_bets = bets.loc[team_odds]
        team_pred = predictions[x:x1]
        team_bets['Model Prediction'] = team_pred
        team_bets['Bet'] = 100
        no_win = team_bets['Model Prediction'] != 'Win' 
        team_bets.loc[no_win,'Bet'] = 0
        team_bets['Profit/Loss'] = 0
        profit_loss = []
        for index,stats in team_bets.iterrows():
            if stats['Model Prediction'] == 'Loss/Draw':
                profit_loss.append(0)
            elif stats['Model Prediction'] == 'Win':
                if (stats['HomeTeam'] == team) & (stats['FTR'] == 'H'):
                    profit_loss.append(stats['B365H'] * 100 - 110)
                elif (stats['AwayTeam'] == team) & (stats['FTR'] == 'A'):
                    profit_loss.append(stats['B365A'] * 100 - 110)
                else:
                    profit_loss.append(-110)
        team_bets['Profit/Loss'] = profit_loss
        teams.append(team)
        total_bets.append(len(team_bets.loc[team_bets['Bet'] == 100]))
        money_made_or_lost.append(team_bets['Profit/Loss'].sum())
        x += 38
        x1 += 38
    results = dict(zip(teams,zip(total_bets,money_made_or_lost)))
    betting_results = pd.DataFrame.from_dict(results, orient='index',columns=['Total Bets Placed', 'Profit/Loss'])
    return betting_results

In [220]:
betting_results = did_I_make_money(team_list,y_pred,bets)
betting_results.head(20)

,Total Bets Placed,Profit/Loss
Arsenal,24,-366.0
Bournemouth,8,361.0
Burnley,9,1230.0
Chelsea,27,403.0
Crystal Palace,8,-69.0
Everton,12,120.0
Hull,4,541.0
Leicester,7,40.0
Liverpool,20,-376.0
Man City,19,111.0


In [221]:
betting_results.sum()

Total Bets Placed     231.0
Profit/Loss          3970.0
dtype: float64